### 2. (torch)  squeeze() 与 unsqueeze()

- squeeze() 压缩一个维度，仅当当前维度为1，否则不做修改
- unsqueeze(num) 增加一个维度，num可以为负数，表示在倒数第num个维度增加一个维度

In [4]:
import torch

x = torch.randn(3, 1, 2)
y = x.squeeze(0)  # not modify
z = x.squeeze(1)  # modified
w = z.unsqueeze(-2)
x.shape, y.shape, z.shape, w.shape

(torch.Size([3, 1, 2]),
 torch.Size([3, 1, 2]),
 torch.Size([3, 2]),
 torch.Size([3, 1, 2]))

### 3. (torch) nn.ReLU()
    相比于max(0, value)，nn.ReLU()可以用于处理批量数据

In [12]:
import torch
import torch.nn as nn

m = torch.nn.ReLU()
x = torch.randn(5)
y = m(x)
x, y

(tensor([-0.3862,  0.3717,  1.3680,  0.5028,  0.0278]),
 tensor([0.0000, 0.3717, 1.3680, 0.5028, 0.0278]))

### 4 `linalg.norm()`计算矩阵的二范式值
    [linalg.norm()](https://pytorch.org/docs/stable/linalg.html?highlight=norm#torch.linalg.norm) 计算矩阵二范式，返回值是一个数

#### 4.1 P范数的一般形式
若 $x=[x_1, x_2, ..., x_n]$，则
$$
    \|x\|_p = (|x_1|^p + |x_2|^p + ... + |x_n|^p)^{(\frac1p)}
$$

当p取0, 1, ∞时，有以下三种情形：
- p=1，一范式：$\|x\|_1 = (|x_1| + |x_2| + ... + |x_n|)$
- p=2，二范式：$ \|x\|_2 = (|x_1|^2 + |x_2|^p + ... + |x_n|^2)^{(\frac12)}$
- p=∞，三范式：$\|x\|_∞ = max(|x_1| + |x_2| + ... + |x_n|)$

#### 4.2 `linalg.norm()` 计算矩阵的二范式，返回值是一个数

In [5]:
import torch

data = torch.ones(2, 2)
data

tensor([[1., 1.],
        [1., 1.]])

##### 4.2.1 linalg.norm() 返回值是二维矩阵的二范式

In [32]:
norm_data = torch.linalg.norm(data)  # torch.linalg.norm(data, p=2, ord=None)
result1 = data / norm_data
norm_data, result1

(tensor(2.),
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000]]))

##### 4.2.2 linalg.norm(data, ord=1) 不是返回二维矩阵的一范式！！！ 而是 max(sum(abs(data), dim=0)

In [3]:
norm_data = torch.linalg.norm(data, ord=1)
result1 = data / norm_data
norm_data, result1

(tensor(2.),
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000]]))

In [5]:
norm_data = max(torch.sum(abs(data), dim=0))
norm_data

tensor(2.)

#### 4.3 手动计算矩阵的二范式，具有可导性????有问题

In [43]:
datax = torch.full((4, 2), 2.)
datax, datax.shape

(tensor([[2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.]]),
 torch.Size([4, 2]))

In [44]:
datax.pow(2)

tensor([[4., 4.],
        [4., 4.],
        [4., 4.],
        [4., 4.]])

In [45]:
datax.pow(2).mean(1, keepdim=True)

tensor([[4.],
        [4.],
        [4.],
        [4.]])

### 5. `nn.functional.normalize() ` 返回沿某个dim维度的二范式矩阵
    dim=0，表示在第一个维度上作二范式，如果keepdim=True，将在第一维度压缩成大小为1

#### 5.1 nn.functional.normalize(input, p, dim, eps) 参数细节

normalize(`input`: Tensor, `p`: float = 2, `dim`: int = 1, `eps`: float = 1e-12, `out`: Optional[Tensor] = None) -> Tensor
- `input`: input tensor of any shape
- `p` (float): the exponent value in the norm formulation. Default: 2
- `dim` (int): the dimension to reduce. Default: 1
- `eps` (float): small value to avoid division by zero. Default: 1e-12
- `out` (Tensor, optional): the output tensor. If :attr:`out` is used, this operation won't be differentiable.

##### 注意：可用于二维矩阵的规范化（大于维度2，则仅用于沿某个维度的二范式计算）

In [8]:
import torch
import torch.nn.functional as fn

data3 = torch.ones(2, 4)
data3

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])

#### 5.2 dim=1（默认值），沿着第二个维度计算二范式矩阵
- norm = sqrt(1^2 + 1^2 + 1^2 + 1^2) = 2
- result = data / norm

In [12]:
result3 = fn.normalize(data3, p=2, dim=1)
result3

tensor([[0.5000, 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5000]])

#### 5.3 dim=0, 沿着第一个维度上计算二范式矩阵
- norm0 = sqrt(1^2) = 1
- norm1 = sqrt(1^2) = 1
- norm2 = sqrt(1^2) = 1
- norm3 = sqrt(1^2) = 1
- result = data / [norm0, norm1, norm2, norm3]

In [14]:
result4 = fn.normalize(data3, p=2, dim=0)  # 第一个维度
result4

tensor([[0.7071, 0.7071, 0.7071, 0.7071],
        [0.7071, 0.7071, 0.7071, 0.7071]])

### 6. (torch) Variable

Variable是对Tensor的一个封装，可以理解为含有计算图的Tensor。每个Variable都有三个属性：
- Varibale的Tensor本身的`.data`
- 对应Tensor的梯度`.grad`
- Variable是通过什么方式得到的`.grad_fn`


#### 6.1 Tensor 与 Variable
    Varibale(tesnor, requires_grad=True) 默认情况下不求梯度的，如果要求梯度，需要说明。

In [15]:
import torch
from torch.autograd import Variable

tensor = torch.FloatTensor([[1, 2], [3, 4]])
tensor

tensor([[1., 2.],
        [3., 4.]])

In [16]:
variable = Variable(tensor, requires_grad=True)
variable

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

#### 6.2 Variable 求梯度
    Variable计算时，会生成计算图，将所有的计算节点都连接起来。在进行误差反向传递时，一次性将所有Variable梯度都计算出来。

In [25]:
from torch.autograd import Variable
import torch

x_tensor = torch.randn(5,5)
y_tensor = torch.ones(5,5)

x = Variable(x_tensor,requires_grad=True) 
y = Variable(y_tensor,requires_grad=True)
z = torch.sum(x + y**2)

z.backward()

z.data, z.grad_fn, x.grad, y.grad

(tensor(28.5746),
 tensor([[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]),
 tensor([[2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.]]))

### 7. torch的排序和比较

#### 7.1 torch的排序
    `sort()`返回从小到大的序列和元素在原序列的位置索引

In [1]:
import torch

arr = [1, 3, 5, 2, 4, 6]
array = torch.tensor(arr)
y, i = torch.sort(array)
y, i

(tensor([1, 2, 3, 4, 5, 6]), tensor([0, 3, 1, 4, 2, 5]))

#### 7.2 torch的比较
- `gt` (greater than) 大于
- `lt` (less than) 小于
- `eq` (equal) 等于
- `ge` (greater and equal) 大于等于
- `le` (less and equal) 小于等于

布尔类型转换为浮点型： result.float() 返回 [1.0, 0.0, ..., 0.0]

布尔类型转换为整型： result.int() 返回 [0, 0, 0, 1, 1]

In [46]:
mask, mask.float(), mask.int()

(tensor([False, False, False,  True,  True]),
 tensor([0., 0., 0., 1., 1.]),
 tensor([0, 0, 0, 1, 1], dtype=torch.int32))

#### 10.3 torch的求和 

In [48]:
torch.sum(mask.float())

tensor(2.)